# Modeling and Evaluation

## Objectives

* **Answer business requirement 2**:  
    * The client wants a system that can **predict whether a cherry leaf is healthy or affected by mildew**.

## Inputs

* `inputs/mildew_dataset/cherry-leaves/train`
* `inputs/mildew_dataset/cherry-leaves/test`
* `inputs/mildew_dataset/cherry-leaves/validation`
* Image shape embedding from previous notebook

## Outputs

* Visual class distribution in each split
* Augmented training images for generalization
* Mapping of class indices to labels
* CNN model: training, validation, and saving
* Training history plot (accuracy/loss)
* Evaluation on test set
* Random prediction from test images

## Additional Comments | Insights | Conclusions

* This notebook develops and evaluates the mildew classification model


---

# Import regular packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

---

# Set Working Directory

In [ ]:
cwd= os.getcwd()
cwd

In [ ]:
os.chdir('d:\\vscode-projects\\mildew-detector-v1')
print("You set a new current directory")

In [ ]:

work_dir = os.getcwd()
work_dir

---

## Set input directories

Set train, validation and test paths

In [5]:
my_data_dir = os.path.join('inputs', 'mildew_dataset', 'cherry-leaves')
train_path = os.path.join(my_data_dir, 'train')
val_path = os.path.join(my_data_dir, 'validation')
test_path = os.path.join(my_data_dir, 'test')

## Set output directory

In [ ]:
version = 'v1'
file_path = os.path.join('outputs', version)

if 'outputs' in os.listdir(work_dir) and version in os.listdir(os.path.join(work_dir, 'outputs')):
    print('Old version is already available, create a new version.')
else:
    os.makedirs(file_path)

## Set labels

In [ ]:
labels = os.listdir(train_path)
print(f"Project Labels: {labels}")

## Set image shape

In [ ]:
## Import saved image shape embedding
import joblib

version = 'v1'
image_shape = joblib.load(os.path.join('outputs', version, 'image_shape.pkl'))
image_shape

---

# Number of images in train, test and validation data

In [ ]:
data = {'Set': [], 'Label': [], 'Frequency': []}
folders = ['train', 'validation', 'test']

for folder in folders:
    for label in labels:
        folder_path = os.path.join(my_data_dir, folder, label)
        count = len(os.listdir(folder_path))
        data['Set'].append(folder)
        data['Label'].append(label)
        data['Frequency'].append(count)
        print(f"* {folder} - {label}: {count} images")

df_freq = pd.DataFrame(data)

print("\n")
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))
sns.barplot(
    data=df_freq,
    x='Set',
    y='Frequency',
    hue='Label',
    palette='Set2'
)
plt.title("Image Count per Dataset Split")
plt.ylabel("Number of Images")
plt.tight_layout()
plt.show()

---

# Image data augmentation

---

### ImageDataGenerator

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

* ### Initialize ImageDataGenerator

In [14]:
augmented_image_data = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

* ### Augment training image dataset

In [ ]:
batch_size = 20  # Set batch size

train_set = augmented_image_data.flow_from_directory(
    train_path,
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

train_set.class_indices

* ### Augment validation image dataset

In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_path,
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

validation_set.class_indices

* ### Augment test image dataset

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_set.class_indices

## Plot augmented training image

In [ ]:
# Plot a few augmented training images
for _ in range(3):
    img_batch, label_batch = next(train_set)
    print(img_batch.shape)  # e.g. (20, 256, 256, 3)
    plt.imshow(img_batch[0])
    plt.axis('off')
    plt.title(f"Label: {int(label_batch[0])}")
    plt.show()

## Save class_indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=os.path.join(file_path, 'class_indices.pkl'))

---

# Model creation

---

## ML model

* ### Import model packages

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

* ### Model 

In [48]:
def create_tf_model():
    model = Sequential()

    model.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu', input_shape=image_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

* ### Model Summary 

In [ ]:
create_tf_model().summary()

* ### Early Stopping 

In [50]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3)

## Fit model for model training

In [ ]:
model = create_tf_model()

history = model.fit(
    train_set,
    epochs=25,
    validation_data=validation_set,
    callbacks=[early_stop],
    verbose=1
)

## Save model

In [53]:
model.save('outputs/v1/mildew_detector_model.keras')

---

# Model Performace

---

## Model learning curves

In [ ]:
# Plot accuracy and loss curves
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(file_path, 'learning_curves.png'), dpi=150)
plt.show()

## Model Evaluation

Load saved model

In [55]:
from keras.models import load_model
model = load_model('outputs/v1/mildew_detector_model.keras')

Evaluate model on test set

In [ ]:
test_loss, test_accuracy = model.evaluate(test_set, verbose=1)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

### Save evaluation

In [ ]:
evaluation = {
    'test_loss': test_loss,
    'test_accuracy': test_accuracy
}

joblib.dump(value=evaluation, filename=os.path.join(file_path, 'evaluation.pkl'))

## Predict on new data

Load a random image as PIL" for two images (one from each label)

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

pointers = [13, 17]
pil_images = []

# Load images
for label, pointer in zip(labels, pointers):
    image_path = os.path.join(test_path, label, os.listdir(os.path.join(test_path, label))[pointer])
    img = image.load_img(image_path, target_size=image_shape, color_mode='rgb')
    pil_images.append((img, label))  # store with label

# Plot side-by-side
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

for i, (img, label) in enumerate(pil_images):
    axes[i].imshow(img)
    axes[i].axis('off')
    axes[i].set_title(f"Label: {label}")

plt.tight_layout()
plt.show()

Convert both PIL images to array and preprocess

In [ ]:
my_images = []

for img, _ in pil_images:
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0) / 255  # Normalize
    my_images.append(arr)

# Stack into a single array with shape (2, height, width, 3)
my_images = np.vstack(my_images)
print(my_images.shape)

Predict probabilities for both images

In [ ]:
pred_proba_array = model.predict(my_images, verbose=0).flatten()

# Map index back to label
target_map = {v: k for k, v in train_set.class_indices.items()}

# Loop over predictions
for i, prob in enumerate(pred_proba_array):
    pred_class = target_map[int(prob > 0.5)]

    # Adjust probability if predicted class is class 0
    if pred_class == target_map[0]:
        prob = 1 - prob

    print(f"Image {i+1} — Predicted class: {pred_class}, Confidence: {prob:.4f}")

---